# Run xpander.ai Agent with Custom Handler

This notebook demonstrates how to initialize, wrap, and interact with an xpander.ai agent using a custom Python handler (`MyAgent`).
It includes loading credentials, listing tools, starting a chat, and printing message history.

In [3]:
import json
from xpander_sdk import XpanderClient, Agent
from my_agent import MyAgent
from dotenv import load_dotenv

b'{"stdout":"bG9hZGluZyBhZ2VudCBmOTFmNmViNy0zZWFkLTQzM2MtOWQ2OC05OGQ0NzhiYjNkYWYK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCBmOTFmNmViNy0zZWFkLTQzM2MtOWQ2OC05OGQ0NzhiYjNkYWYgd2l0aCBleGVjdXRpb24gYTI0YWZhMjYtNjQxYS00NGYxLTkxZDktMDZjOGUzMmU1ODQyCg=="}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCBmOTFmNmViNy0zZWFkLTQzM2MtOWQ2OC05OGQ0NzhiYjNkYWYgd2l0aCBleGVjdXRpb24gZjNlZDczZGItNDI2My00ZWI5LTg4ZDQtZWJiOTVkZDQ0ZDY5Cg=="}\n'


## Load Environment & Agent Config
Loads credentials from `.env` and agent metadata from `xpander_config.json`.

In [4]:
load_dotenv()

with open('xpander_config.json', 'r') as config_file:
    xpander_config: dict = json.load(config_file)

## Initialize Agent Instance
Connects to xpander.ai BaaS (Backend-as-a-service) and loads the agent specified in the config.

In [ ]:
xpander_client = XpanderClient(api_key=xpander_config.get("api_key"))
xpander_agent: Agent = xpander_client.agents.get(
    agent_id=xpander_config.get("agent_id"))

In [6]:
print(f"Loaded Agent: {xpander_agent.name}")

Loaded Agent: Coding Agent


## List Agent Tools
Displays all tools the agent can use, with descriptions and parameter details.

In [8]:
tools = xpander_agent.get_tools()
tools_table = []

for tool in tools:
    func = tool.get("function", {})
    name = func.get("name", "")
    description = func.get("description", "").strip().replace(
        "\n", " ")[:200] + "..."
    params = func.get("parameters", {}).get("properties", {})
    formatted_params = json.dumps(params)
    tools_table.append([name, description, formatted_params])

print("Agent's attached tools:", tools)

Agent's attached tools: [{'type': 'function', 'function': {'name': 'GitHubPullManagementUpdatePullRequestBranch', 'description': 'Updates the pull request branch by merging the latest changes from the base branch into the pull request branch, keeping it in sync with the upstream repository. This operation requires that the repository owner, repository name, pull request number, and the expected current HEAD SHA are provided. It is a crucial step in maintaining an up-to-date branch for continued development and eventual merging. If any of the required fields is missing, run GitHubPullManagementListCommitsForPullRequest before proceeding to acquire the latest commit SHA from the pull request branch. By doing so, you ensure that the branch update is based on the most recent commit data, thus securing a consistent integration flow.', 'parameters': {'type': 'object', 'properties': {'bodyParams': {'type': 'object', 'properties': {'expected_head_sha': {'type': 'string', 'description': 'The ex

In [14]:
!pip install tabulate
import tabulate

In [17]:
print(tabulate.tabulate(tools_table, headers=[
      "Name", "Description", "Params JSON"], tablefmt="grid"))

+------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Wrap Agent with Custom Handler
Creates a custom agent interface (`MyAgent`) that controls how tasks are handled.

In [18]:
agent = MyAgent(xpander_agent)

## Start a Chat with the Agent
Sends a first message (`Hi!`) and runs the custom handler loop.

In [19]:
thread = await agent.chat("Hi!")

2025-06-27 15:35:38.235 | INFO     | my_agent:chat:79 - 🧠 Adding task to a new thread
2025-06-27 15:35:46.617 | INFO     | my_agent:_agent_loop:114 - 🪄 Starting Agent Loop
2025-06-27 15:35:48.200 | INFO     | my_agent:_agent_loop:120 - --------------------------------------------------------------------------------
2025-06-27 15:35:48.200 | INFO     | my_agent:_agent_loop:121 - 🔍 Step 1
2025-06-27 15:35:50.839 | INFO     | providers.llms.openai.async_client:invoke_model:87 - 🔄 Model response received in 2.53 s
2025-06-27 15:35:50.840 | INFO     | providers.llms.openai.async_client:invoke_model:93 - 🔄 Tool call function name: xpfinish-agent-execution-finished
2025-06-27 15:35:54.611 | INFO     | my_agent:_agent_loop:178 - ✅ xpfinish-agent-execution-finished
2025-06-27 15:35:54.612 | INFO     | my_agent:_agent_loop:180 - 🔢 Step 1 tokens used: 3511 (output: 46, input: 3465)
2025-06-27 15:35:54.914 | INFO     | my_agent:_agent_loop:186 - ✨ Execution duration: 8.30 s
2025-06-27 15:35:54.918

## View Execution Result (First Message)
After sending the first message, we'll fetch the agent's execution result. This includes:
- `status`: Current state of the task (`PENDING`, `EXECUTING`, `PAUSED`, `ERROR`, or `COMPLETED`)
- `result`: The agent's final answer or error message

In [20]:
execution_result = xpander_agent.retrieve_execution_result()
print("status: ", execution_result.status.value)
print("result: ", execution_result.result)

status:  COMPLETED
result:  Hello! How can I assist you today?


## View Agent Message History
Displays messages from the current conversation thread in a readable table.

In [22]:
print("The agent's thread:")
messages = xpander_agent.messages

table = []
for msg in messages:
    role = msg.get("role", "")
    content = msg.get("content", "")
    content_preview = content.strip().replace("\n", " ")
    content_preview = (
        content_preview[:100] + "...") if len(content_preview) > 100 else content_preview
    tool_calls = msg.get("tool_calls", "")
    tool_calls_json = json.dumps(tool_calls) if tool_calls else ""
    table.append([role, content_preview, tool_calls_json])

print(tabulate.tabulate(table, headers=[
      "Role", "Content", "Tool Calls"], tablefmt="grid"))

The agent's thread:
+-----------+---------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Role      | Content                                                                                                 | Tool Calls                                                                                                                                                                                                                                                             |
+===========+=========================================================================================================+===========================================================================================

## Continue the Chat
Sends a follow-up message in the same thread to preserve context.

In [23]:
await agent.chat("What can you do ?", thread)

2025-06-27 15:36:12.233 | INFO     | my_agent:chat:76 - 🧠 Adding task to existing thread: 34a82c9a-2a82-4ec2-ac80-089c5c1671da
2025-06-27 15:36:15.508 | INFO     | my_agent:_agent_loop:114 - 🪄 Starting Agent Loop
2025-06-27 15:36:15.714 | INFO     | my_agent:_agent_loop:120 - --------------------------------------------------------------------------------
2025-06-27 15:36:15.714 | INFO     | my_agent:_agent_loop:121 - 🔍 Step 1
2025-06-27 15:36:19.065 | INFO     | providers.llms.openai.async_client:invoke_model:87 - 🔄 Model response received in 3.22 s
2025-06-27 15:36:19.066 | INFO     | providers.llms.openai.async_client:invoke_model:93 - 🔄 Tool call function name: xpfinish-agent-execution-finished
2025-06-27 15:36:23.371 | INFO     | my_agent:_agent_loop:178 - ✅ xpfinish-agent-execution-finished
2025-06-27 15:36:23.372 | INFO     | my_agent:_agent_loop:180 - 🔢 Step 1 tokens used: 3624 (output: 150, input: 3474)
2025-06-27 15:36:23.710 | INFO     | my_agent:_agent_loop:186 - ✨ Executio

'34a82c9a-2a82-4ec2-ac80-089c5c1671da'

## View Execution Result (Follow-Up)

This displays the execution result after the second chat message.  
Use it to verify if the agent completed the task or encountered an issue.

In [24]:
execution_result = xpander_agent.retrieve_execution_result()
print("status: ", execution_result.status.value)
print("result: ", execution_result.result)

status:  COMPLETED
result:  Hello! I can help you with a variety of tasks, including:

- Answering questions and providing explanations on many topics
- Generating and editing text, such as emails, summaries, or creative writing
- Creating and editing images based on your descriptions
- Assisting with code generation, debugging, and code reviews
- Managing GitHub pull requests, including creating, updating, and commenting on them
- Downloading files from the internet and reading local files (in a sandboxed environment)

Let me know what you need help with!
